# Extraction of SWR from CA1 recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [10]:
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from ipyfilechooser import FileChooser
from IPython.display import display
import ipywidgets as widgets


%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer, EventList, InMemoryEventSource
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

In [4]:
dpath = ""
try:
    %store -r dpath
except:
    print("data path not in strore")
    dpath = os.path.expanduser("~")

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>OpenEphys Folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/session5', filename='', title='<b>OpenEphys Fo…

In [5]:
suffix=''#'_AB'
sep = -1
animalIDPos = -3
dirPathComponents = os.path.normpath(dpath).split(os.sep)
rootPath = os.path.join('/',*dirPathComponents[:sep])
folder_base = os.path.join(*dirPathComponents[sep:])
mice = "GreenLinesOK" #dirPathComponents[animalIDPos]
os.chdir(rootPath)
print(rootPath)
print(folder_base)
print(mice)

/Users/mb/Documents/Syntuitio/AudreyHay/PlanB
session5
GreenLinesOK


In [6]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

filename = os.path.join(folder_base,f'LFPwake0{suffix}.npy')
filename3 = os.path.join(folder_base,f'LFPwakeremoved{suffix}.npy')
filename2 = os.path.join(folder_base,'RawDataChannelExtractedDS.npy')
EMGbooleaninput = os.path.join(folder_base,f'EMGframeBoolean{suffix}.pkl')


EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremoved = np.load(filename3, mmap_mode= 'r')
All = np.load(filename2, mmap_mode= 'r')

try:
    Channels = os.path.join('',f'LFPChannels_perMice.xlsx')
    allchannels = pd.read_excel(Channels)
    PFCch1=int(allchannels[mice][0].split(',')[0])
    PFCch2=int(allchannels[mice][0].split(',')[1])
    CA1ch1=int(allchannels[mice][2].split(',')[0])
    CA1ch2=int(allchannels[mice][2].split(',')[1])
except FileNotFoundError as e:
    print(color.BOLD + color.YELLOW)
    print(f"File {os.path.join(folder_base,f'LFPChannels_perMice.xlsx')} not found!")
    print("probably you are not Aurélie... or the path to access it is wrong.")
    print("In the first case, make sure the mapping is rightfully setup in the curent cell")
    print("In the second case, you can play with the 'sep' variable of cell 3, or directly change the path")
    print(color.END)
    PFCch1=19
    PFCch2=20
    CA1ch1=21
    CA1ch2=22

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
CA1wakeremoved = LFPwakeremoved[:,CA1ch1]-LFPwakeremoved[:,CA1ch2] 

# Band pass filter
        SWR: 120-200 Hz

In [7]:
# Filtre parameter:
f_lowcut = 120.
f_hicut = 200.
fs = 1000
nyq = 0.5 * fs
N = 6                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_CA1 = signal.filtfilt(b, a, CA1)
filt_CA1wakeremoved = signal.filtfilt(b, a, CA1wakeremoved)

# Plot
times = np.arange(0, CA1.size/fs, 1./fs)
#timesmin = np.arange(0, CA1.size/fs/60, 1./fs/60)
#fig, ax = plt.subplots()
#ax.plot(timesmin, filt_CA1)

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [8]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(120, 200, 80)
widths = w*fs / (2*freq*np.pi)
CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation
absCA1NWcwt = np.absolute(CA1NWcwt)
proj_CA1NWcwt = np.sum(absCA1NWcwt, axis = 0)/80
sdproj_CA1cwt = np.std(proj_CA1NWcwt)
sd3proj_CA1cwt = sdproj_CA1cwt*3
sd10proj_CA1cwt = sdproj_CA1cwt*10
sd8proj_CA1cwt = sdproj_CA1cwt*8
sd7proj_CA1cwt = sdproj_CA1cwt*7
sd05proj_CA1cwt = sdproj_CA1cwt*0.5

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_46938/3157878907.py:5: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)


Second on the signal for which wake times have been zeroed

In [9]:
# Conservative boolean filtering of CA1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fCA1wake0C = filt_CA1.copy()
fCA1wake0C[BooleanCons] = 0
CA1wake0C = CA1.copy()
CA1wake0C[BooleanCons] = 0
# Liberal boolean filtering of CA1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fCA1wake0L = filt_CA1.copy()
fCA1wake0L[BooleanLib] = 0
CA1wake0L = CA1.copy()
CA1wake0L[BooleanLib] = 0

# Computation of CWT
CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absCA1W0Ccwt = np.absolute(CA1cwtWake0cons)
proj_CA1W0Ccwt = np.sum(absCA1W0Ccwt, axis = 0)/80
absCA1W0Lcwt = np.absolute(CA1cwtWake0lib)
proj_CA1W0Lcwt = np.sum(absCA1W0Lcwt, axis = 0)/80

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_46938/2403058388.py:15: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_46938/2403058388.py:16: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)


## Extracting SWRs and determining main properties 

First extraction of SWR peaks, initiation, end and width

In [11]:
# 10 sd threshold
peaks, properties = find_peaks(proj_CA1W0Lcwt, prominence=1, width=20, height=sd8proj_CA1cwt) #AB detection with 8*SD // Audrey's detection=3*SD
properties["prominences"], properties["widths"]

# SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CA1W0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

Display subset

Second extraction of main frequency and power 

In [12]:
projMaxP_cwtmg = np.max(CA1cwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(CA1cwtWake0lib, axis = 0) + 120
projMaxP_cwtmg.shape

nb_SWR = len(peaks)
data = np.zeros((nb_SWR,4))

for tt in np.arange(nb_SWR):
    SWR_start = int(SWR_prop[3,tt])
    SWR_stop = int(SWR_prop[4,tt])
    SWR_MaxP = projMaxP_cwtmg[SWR_start:SWR_stop]
    SWR_MaxF = projMaxF_cwtmg[SWR_start:SWR_stop]
    data[tt, 0] = max(SWR_MaxF).round()
    data[tt, 1] = max(SWR_MaxP).round()
    data[tt, 2] = round(sum(SWR_MaxF)/len(SWR_MaxF))
    data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))

param_SWR = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSWR_prop = SWR_prop.transpose()
pd_prop_SWR = pd.DataFrame(tSWR_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
pd_tokeep = pd.DataFrame(np.ones(nb_SWR).astype(bool), columns = ['toKeep'])
All_SWR = pd.concat([pd_tokeep,pd_prop_SWR, param_SWR], axis=1)

SWR_peak = peaks
SWR_start = SWR_prop[3,:].astype(int)
SWR_end = SWR_prop[4,:].astype(int)

/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_46938/2193912474.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(SWR_MaxP).round()
/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_46938/2193912474.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))
/var/folders/2_/fpwjwx512255f113wntjtndm0000gn/T/ipykernel_46938/2193912474.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))


### Store the results in All_SWR_prop pd dataframe and save as pkl/csv for post processing.

End of Notebook. 

In [13]:
filename2 = os.path.join(folder_base,f'SWRproperties{suffix}.pkl')
filename3 = os.path.join(folder_base,f'SWRproperties{suffix}.csv')
All_SWR.to_pickle(filename2)
All_SWR.to_csv(filename3, sep = ',')

combined = np.stack([CA1, filt_CA1, proj_CA1W0Ccwt, proj_CA1W0Lcwt], axis = 1)
filenameC = os.path.join(folder_base,f'SignalCA1.npy')
np.save(filenameC, combined)

sample_rate = 1000.
t_start = 0.
# if done and no intention to display for assessment
#%reset
#plt.close('all')

Load previous SWR detection (only if wanted)

In [ ]:
"""
OldSWSdetection=folder_base / f'SWRproperties.csv'
SWR_prop = pd.read_csv(OldSWSdetection, index_col=0)            
pd_prop_SWR=All_SWR.iloc[:, :5]
param_SWR=All_SWR.iloc[:, 5:9]
SWR_prop=SWR_prop.values
SWR_peak = np.transpose(SWR_prop[:,0].astype(int))
SWR_start = np.transpose(SWR_prop[:,3].astype(int))
SWR_end = np.transpose(SWR_prop[:,4].astype(int))
"""

### Display and asses SWRs

#### ephys viewer to check SWR detection

In [30]:
%gui qt
app = mkQApp()

#Create one data source with 3 event channel
all_events = []
conditions = ['All','Good','Bad']
for c,cond in enumerate(conditions):
    match cond:
        case 'All':
            selection = "All_SWR['toKeep'] | ~All_SWR['toKeep']"
        case 'Good':
            selection = "All_SWR['toKeep']"
        case 'Bad':
            selection = "~All_SWR['toKeep']"
    ev_times = All_SWR.loc[pd.eval(selection),'peak time'].values/1000
    ev_labels = [f'SWR {i}'for i in All_SWR[pd.eval(selection)].index]
    all_events.append({ 'time':ev_times, 'label':ev_labels, 'name': conditions[c] })
source_ev = InMemoryEventSource(all_events=all_events)



sample_rate = 1000.
t_start = 0.

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: SWR_peak, 1: SWR_start, 2: SWR_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
view1.auto_scale()

nCh = len(view1.by_channel_params.children())
mult = 4
for ch in range(nCh):
    match ch%mult:
        case 0: # raw traces
            view1.by_channel_params[f'ch{ch}', 'offset'] = 2.5 + 3*int(ch/mult)
            #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.05
            view1.by_channel_params[f'ch{ch}', 'color'] = '#ffffff'
        case 1: # filtered traces
            view1.by_channel_params[f'ch{ch}', 'offset'] = 0.5 + 3*int(ch/mult)
            #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.1
            view1.by_channel_params[f'ch{ch}', 'color'] = '#0055ff'
        case 2: # envelop
            view1.by_channel_params[f'ch{ch}', 'offset'] = 0.5 + 3*int(ch/mult)
            #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.1
            view1.by_channel_params[f'ch{ch}', 'color'] = '#ff5500'
        case 3: # envelop
            view1.by_channel_params[f'ch{ch}', 'offset'] = 0.5 + 3*int(ch/mult)
            #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.1
            view1.by_channel_params[f'ch{ch}', 'color'] = '#ffffff'
view1.params['ylim_max']=3*int((nCh+1)/mult)
view1.params['ylim_min']=0

view2 = EventList(source=source_ev, name='event')


#put this viewer in the main window
win.add_view(view1)
win.add_view(view2, location='bottom',  orientation='horizontal')

#Run
win.show()

debug True
QT_MODE PySide6
refresh duration for  4.792213439941406e-05 s
refresh duration for  4.506111145019531e-05 s
refresh duration for event 3.814697265625e-06 s


qt.pointer.dispatch: skipping QEventPoint(id=0 ts=0 pos=0,0 scn=377.997,251.63 gbl=377.997,251.63 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-377.997,-251.63 last=-377.997,-251.63 Δ 377.997,251.63) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=540.002,201.949 gbl=540.002,201.949 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-540.002,-201.949 last=-540.002,-201.949 Δ 540.002,201.949) : no target window


refresh duration for  0.0002810955047607422 s


#### Select and deselect SWRs

In [33]:
def clicked(arg):
    selectedEvent = view2.list_widget.currentItem().text()
    selectedSWR = int(selectedEvent.split('SWR')[1])
    #print(selectedSWR)
    match arg.description:
        case 'Keep SWR':
            All_SWR.loc[selectedSWR,'toKeep']=True
            print(f'SWR {selectedSWR} restored')
        case 'Discard SWR':
            All_SWR.loc[selectedSWR,'toKeep']=False
            print(f'SWR {selectedSWR} discarded')
    #Create one data source with 3 event channel
    all_events = []
    conditions = ['All','Good','Bad']
    for c,cond in enumerate(conditions):
        match cond:
            case 'All':
                selection = "All_SWR['toKeep'] | ~All_SWR['toKeep']"
            case 'Good':
                selection = "All_SWR['toKeep']"
            case 'Bad':
                selection = "~All_SWR['toKeep']"
        ev_times = All_SWR.loc[pd.eval(selection),'peak time'].values/1000
        ev_labels = [f'SWR {i}'for i in All_SWR[pd.eval(selection)].index]
        all_events.append({ 'time':ev_times, 'label':ev_labels, 'name': conditions[c] })
    source_ev = InMemoryEventSource(all_events=all_events)
    view2.source = source_ev
    view2.refresh_list(view2.combo.currentIndex())

button_Good = widgets.Button(description = 'Keep SWR')   
button_Good.on_click(clicked)

button_Bad = widgets.Button(description = 'Discard SWR')   
button_Bad.on_click(clicked)

display(button_Good, button_Bad)



Button(description='Keep SWR', style=ButtonStyle())

Button(description='Discard SWR', style=ButtonStyle())

refresh duration for  0.0003750324249267578 s
refresh duration for  0.0003311634063720703 s
5
SWR 5 discarded


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=581.361,419.059 gbl=581.361,419.059 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-581.361,-419.059 last=-581.361,-419.059 Δ 581.361,419.059) : no target window


refresh duration for  0.0002760887145996094 s
5
SWR 5 restored
save_all_settings


In [ ]:
# Set up selector object
discrete_slider = pn.widgets.DiscreteSlider(
    name="Choose unit", 
    options=[i for i in a.index],
    value=a.index[0]
)

drop_unit_button = pn.widgets.Button(name='Drop this unit! >', button_type='primary')
keep_unit_button = pn.widgets.Button(name='Keep this unit!')
next_unit_button = pn.widgets.Button(name='Next unit > ', button_type='primary')
previous_unit_button = pn.widgets.Button(name='< Previous unit')

text_input = pn.widgets.TextInput(value='', width=300)

# Define a callback function for the button
def dropunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} dropped!'
    number = selected_value
    if number in unit_to_keep:
        unit_to_keep.remove(number)
    nextunit_callback(event)
    text_input.value = text

# Define a callback function for the button
def keepunit_callback(event):
    selected_value = discrete_slider.value
    text=f'Unit n°{selected_value} kept!'
    number = selected_value
    if number not in unit_to_keep:
        unit_to_keep.append(number)
        unit_to_keep.sort()
    text_input.value = text

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

drop_unit_button.on_click(dropunit_callback)
keep_unit_button.on_click(keepunit_callback)
next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)